### Get Input File & Setup

In [1]:
table=[
    [1,1,1,1,1],
    [1,1,1,0,1],
    [1,0,1,1,1],
    [0,1,1,1,1],
    [0,1,1,0,1],
    [1,0,1,0,0],
    [0,0,0,1,1],
    [0,0,0,0,0]
]

B = {
    'A','B','C','D','E'
}

FactorHash = {
    'A': 0,
    'B': 1,
    'C': 2,
    'D': 3,
    'E': 4
}

NegFactorSet = {}

### Perform Alg on File

In [2]:
#STEP 0

# FactorTable is a hashtable/dictionary of all factors, factorA -> index | key -> value
# For everything in W, we will have to test this
# Over every factor, we are collecting the set of factors that are possible effects, this set is W
# B: set of all factors, main_table: C
def step0(main_table, B, FactorTable, NegFactorSet):
    print("mainTable:")
    print(main_table)
    W = []
    for factor in B:
        print("Looking at " + factor)
        #(2)
        if rowDuplicity(main_table, FactorTable[factor]):
            print(factor + " failed row duplicity")
            continue
        #(3)
        elif isNeg(FactorTable[factor], NegFactorSet):
            print(factor + " failed isNeg")
            continue
        W.append(factor)
    return W

'''
    Check if the same row but negative factor already exists, if not, insert into set, else return 1
    int[i][j] table, i= coincidence, j = factor index
'''
def rowDuplicity(table, factor):
    rowSet = []
    #loop through every row in table
    for coincidence in table:
        modif_coincidence = coincidence.copy()
        modif_coincidence[factor] = 1-modif_coincidence[factor]
        if modif_coincidence in rowSet:
            return 1
        rowSet.append(coincidence)
        
def isNeg(factor, NegFactorSet):
    if factor in NegFactorSet:
        return 1
    return 0

In [3]:
potential_effects = step0(table, B, FactorHash, NegFactorSet)

mainTable:
[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]
Looking at E
Looking at B
B failed row duplicity
Looking at A
A failed row duplicity
Looking at D
D failed row duplicity
Looking at C


### Main Loop

In [4]:
#-- HELPER FUNCTIONS - SELF SET FUNCTIONS --##

def setIn(fauxSet, coincidence):
    #loop through fauxSet
#     print("---------")
#     print(coincidence)
#     print(fauxSet)
#     print("---------")
    for elem in fauxSet:
        if len(elem) == len(coincidence):
#             print("888888")
#             print(elem)
#             print(coincidence)
#             print("888888")
            for index, fac in enumerate(elem):
                #compare each index in fauxElement to coincidence
                if coincidence[index] != fac:
                    break
                if index == len(elem)-1:
                    return True
    return False

def ifPairListsEqual(pairList1, pairList2):
    for pair1 in pairList1:
        exists = False
        for pair2 in pairList2:
            if pair1[0] == pair2[0] and pair1[1] == pair2[1]:
                exists = True
        if not exists:
            return False
    return True

def pairListIn(conditionList, pairList):
    for elem in conditionList:
        if ifPairListsEqual(elem, pairList):
            return True
    return False
        

In [5]:

#-------------------------------------------------------------------------------
#STEP 1
#FOR LOOP
# [[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], 
#  [0, 0, 0, 0, 0]]
# C
# []
# [[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]
# E
# []
#--------------------------------
#STEP 2

from numpy import *

'''
table: int[][] : double array of 1/0
factor: int : index of Z_i 
coincidence: X_k Z_i
modif_coincidence: X_k !Z_i
returns set of rows where coincidence X_k is sufficient for Z_i
'''
def step2(main_table, factor):
    rowSet = []
    for coincidence in table:
        indexList = list(range(0,len(coincidence)))
        indexList.remove(FactorHash[factor])
        if check_sufficient(coincidence, indexList, main_table, factor):
            rowSet.append(coincidence)
    return rowSet
#--------------------------------
#STEP 3

from itertools import permutations
'''

'''
def step3(rowSet, main_table, effect):
    minimally_sufficient_factors = []
        
    for coincidence in rowSet:
        suff_set = []
        prm_list = permutations(range(0,len(coincidence)))
        # for every permutation of coincidence
        for prm in prm_list:
            #remove Z_g
            tmp_prm = list(prm).copy()
            for factor in prm:
                #remove
                tmp_prm.remove(factor)
                #is remaining prm suff?
                if not check_sufficient(coincidence, tmp_prm, table, effect):
                    tmp_prm.append(factor)
            if not setIn(suff_set, tmp_prm) and not(len(tmp_prm)==1 and tmp_prm[0] == FactorHash[effect]):
                suff_set.append(tmp_prm)
                
        for suff in suff_set:
            suff_obj = []
            for index in suff:
                pair = (index, coincidence[index])
                suff_obj.append(pair)
            if not pairListIn(minimally_sufficient_factors, suff_obj):
                minimally_sufficient_factors.append(suff_obj)
                    
    return minimally_sufficient_factors
                    
# returns 1 if sufficient
def check_sufficient(coincidence, prm, table, effect):
    extravar = False
    for row in table:
        var = True
        for index,col in enumerate(row):
            if index in prm:
                if col != coincidence[index]:
                    var = False
                    break
        if var is True:
            extravar = True
            #in order for the original coincidence to be sufficient, its effect had to have been 1
            if row[FactorHash[effect]] == 0:
                return 0
    return extravar



In [12]:
#--------------------------------
#STEP 4
#END OF FOR LOOP -- WE ARE IGNORING THIS STEP FOR EFFICIENCY

#--------------------------------
#STEP 5

def step5(main_table, sufficient_pair_lists, effect):
    for pair_list in sufficient_pair_lists:
        if not check_necessary(main_table, pair_list, effect):
            sufficient_pair_lists.remove(pair_list)
    return sufficient_pair_lists

#--------------------------------
#STEP 6

from itertools import permutations

# def step6(rowSet, main_table, effect)
#     for coincidence in rowSet:
#         prm_list = list(permutations(range(0,coincidence.length)))
#         # for every permutation of coincidence
#         for prm in prm_list:
#             #remove Z_g
#             for factor in prm:
#                 #remove
#                 removeprm()
#                 #is remaining prm suff?
#                 if !check_necessary(table, coincidence, mod_prm, effect):
#                     addbackprm()

#pair[0] is index, pair[1] is value        
def check_necessary(main_table, pair_list, effect):
    for row in main_table:
        if row[FactorHash[effect]] == 1:
            for pair in pair_list:
                if not (pair[1] == row[pair[0]]):
                    return False
    return True

In [16]:
print(table)
for effect in potential_effects:
    rowSet = step2(table,effect)
    sufficientSet = step3(rowSet,table,effect)
    necessarySet = step5(table, sufficientSet, effect)
    #for future: clean sufficient to remove itself (own factor) as well as duplicate
    print(effect)
    print(rowSet)
    print("0----0")
    for pairlist in sufficientSet:
        for pair in pairlist:
            print(pair)
        print("0----0")
    print("===^Sufficient | \/ necessary ===")
    print("0----0")
    for pairlist in necessarySet:
        for pair in pairlist:
            print(pair)
        print("0----0")
    print("BREAK")

[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0], [0, 0, 0, 1, 1], [0, 0, 0, 0, 0]]
E
[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [0, 0, 0, 1, 1]]
0----0
(1, 1)
0----0
===^Sufficient | \/ necessary ===
0----0
(1, 1)
0----0
BREAK
C
[[1, 1, 1, 1, 1], [1, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 1], [0, 1, 1, 0, 1], [1, 0, 1, 0, 0]]
0----0
(0, 1)
0----0
===^Sufficient | \/ necessary ===
0----0
(0, 1)
0----0
BREAK


### Framing